# ESTADO DE DESAGREGACIÓN EN LA FULL RIB

Se deberá por medio de BGPstream conseguir una tabla de ruteo BGP completa (full RIB), para luego inspeccionar cuantas entradas tiene la tabla antes y después de efectuar la máxima agregación posible.

IMPORTANTE: La agregación de prefijos debe tener en cuenta los AS origen. Es decir, prefijos continuos pero originados por diferentes ASes no pueden ser agregados.

Para poder llevar a cabo el ejercicio deberá seleccionar:

* Un colector: podrá ser cualquiera que crea conveniente de la siguiente lista https://bgpstream.caida.org/data.

* Una fecha: deberá usar como día y mes, la fecha de cumpleaños de alguno de los integrantes, y como año, el valor 2015. El colector elegido deberá disponer de información en esa fecha.

In [26]:
import pybgpstream
import pprint
import numpy as np
from itertools import groupby
import pandas as pd

#!pip install ipaddr
import ipaddr, ipaddress

In [5]:
!python script_ej2.py

In [9]:
df = pd.read_csv('rib_ej2.csv', delim_whitespace = 5 )

/Users/mariaflorenciaprado/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df.head()

,as_origen,prefix
0,1251,0.0.0.0/0
1,262476,0.0.0.0/0
2,1251,0.0.0.0/0
3,15169,1.0.0.0/24
4,15169,1.0.0.0/24


In [11]:
df.size

12759132

In [40]:
df['prefix'] = df['prefix'].apply(lambda x : ipaddress.ip_network(str(x))) #Convierto los prefijos a ip type

In [41]:
df.head()

,as_origen,prefix
0,1251,0.0.0.0/0
1,262476,0.0.0.0/0
2,1251,0.0.0.0/0
3,15169,1.0.0.0/24
4,15169,1.0.0.0/24


In [71]:
df_IPv4 = (df.loc[df['prefix'].apply(lambda x : x.version == 4)])
df_IPv4.head()

,as_origen,prefix
0,1251,0.0.0.0/0
1,262476,0.0.0.0/0
2,1251,0.0.0.0/0
3,15169,1.0.0.0/24
4,15169,1.0.0.0/24


In [66]:
df_IPv4.size

12331446

In [72]:
df_IPv6 = (df.loc[df['prefix'].apply(lambda x : x.version == 6)])
df_IPv6.head()

,as_origen,prefix
6165723,1251,::/0
6165724,262476,::/0
6165725,262476,::/0
6165726,14840,::/0
6165727,17465,0:2000::/20


In [67]:
df_IPv6.size

427686

In [70]:
12331446 + 427686 == 12759132 #Verifico

True

In [98]:
rib_IPv4_aggregated = df_IPv4.groupby('as_origen', as_index=False).agg(lambda x: ipaddress.collapse_addresses(x['prefix']))

In [101]:
rib_IPv4_aggregated.as_origen.size

51981

In [116]:
rib_IPv4_aggregated.prefix = rib_IPv4_aggregated.prefix.apply(lambda x : list(x))

In [117]:
rib_IPv4_aggregated.head(10)

,as_origen,prefix
0,5,[189.217.0.0/20]
1,42,[189.201.244.0/23]
2,174,"[188.64.32.0/21, 190.105.156.0/23]"
3,513,[188.184.0.0/16]
4,680,[188.1.0.0/16]
5,1134,"[188.207.32.0/20, 188.207.48.0/21, 188.207.56...."
6,1136,"[188.92.152.0/23, 188.200.0.0/13]"
7,1241,[188.4.0.0/16]
8,1257,[188.65.152.0/21]
9,1435,[190.124.40.0/21]


In [109]:
rib_IPv6_aggregated = df_IPv6.groupby('as_origen', as_index=False).agg(lambda x:  ipaddress.collapse_addresses(x['prefix']))

In [118]:
rib_IPv6_aggregated.prefix = rib_IPv6_aggregated.prefix.apply(lambda x : list(x))

In [119]:
rib_IPv6_aggregated.head()

,as_origen,prefix
0,10000,[2405:ed00::/32]
1,10001,[2403:7c00::/32]
2,10002,[2400:8300::/32]
3,10006,"[2403:9400:1::/48, 2403:9400:2::/48]"
4,10010,"[2401:980::/32, 2401:af80::/33, 2403:1f00::/32..."


In [111]:
rib_IPv6_aggregated.as_origen.size

9772

Inicialmente la tabla tenía 12759132, tras la agregación por un lado de las direcciones IPv4 y por otro IPv6, quedaron 51981 IPv4 y 9772 IPv6 entradas, sumando un total de 61753 entradas